https://ansforge.github.io/annuaire-sante-fhir-documentation/pages/guide/version-1/getting-started/get-api-key.html

In [ ]:
import pandas as pd
import os
import glob
import requests
from datetime import datetime, timedelta
import zipfile
import io

# 1. Construis la variable date_file à aujourd'hui au format: YYYYMMDD, et accole la date de demain au format MMDD
today = datetime.now()
tomorrow = today + timedelta(days=1)
date_file = today.strftime('%Y%m%d') + tomorrow.strftime('%m%d')

# 2. Télécharge le fichier zip
print('Téléchargement du fichier: ' + date_file)
url = f"https://annuaire.sante.fr/web/site-pro/extractions-publiques?p_p_id=abonnementportlet_WAR_Inscriptionportlet_INSTANCE_gGMT6fhOPMYV&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage&_abonnementportlet_WAR_Inscriptionportlet_INSTANCE_gGMT6fhOPMYV_nomFichier=PS_LibreAcces_{date_file}.zip"

# try:
#     response = requests.get(url)
#     response.raise_for_status()  # Lève une exception si la requête a échoué
#     zip_file = zipfile.ZipFile(io.BytesIO(response.content))
#     print(f"Le fichier ZIP a été téléchargé avec succès : {url}")
# except requests.exceptions.HTTPError as err:
#     print(f"Erreur HTTP: {err}")
#     print("Veuillez vérifier l'URL ou le format de la date.")
#     exit()

# 3. Extrais l'archive zip dans le répertoire courant
# zip_file.extractall()
# print("L'archive ZIP a été extraite.")

# 4. Ouvre le fichier texte comme un CSV
file_path = glob.glob("PS_LibreAcces_Personne_activite_*.txt")[0]
df = pd.read_csv(file_path, sep='|', encoding='utf-8', on_bad_lines='skip', dtype=str)

# 5.  Ne charge que les lignes dont la colonne Code profession vaut 10 (= Médecin)
# df = df[df.iloc[:, 9] == 10]
df = df[df['Code profession'] == '10']

# 6. Enregistre au format CSV, séparateur ";", les colonnes spécifiées
# Sélection des colonnes par leur indice (la 2e est à l'indice 1, etc.)
colonnes_a_garder = [1, 3, 5, 7, 8, 16, 17, 24, 26, 28, 29, 31, 32, 35, 37, 40, 42]
df_final = df.iloc[:, colonnes_a_garder]
df_final.to_csv('annuaire-medecins.csv', sep=';', index=False, encoding='utf-8')
print("Le fichier annuaire-medecins.csv a été créé avec succès.")

# Nettoyage
motif_de_recherche = "PS_LibreAcces*"
fichiers_a_effacer = glob.glob(motif_de_recherche)

# Vérifier si des fichiers ont été trouvés avant de continuer
if not fichiers_a_effacer:
    print("Aucun fichier commençant par 'PS_LibreAcces' n'a été trouvé.")
else:
    print(f"Les fichiers suivants vont être effacés :")
    for fichier in fichiers_a_effacer:
        print(f"- {fichier}")
    
    # Parcourir la liste et effacer chaque fichier
    for fichier in fichiers_a_effacer:
        try:
            os.remove(fichier)
            print(f"Fichier '{fichier}' effacé avec succès.")
        except OSError as e:
            print(f"Erreur lors de l'effacement du fichier '{fichier}': {e}")
    
    print("\nOpération terminée.")